In [1]:
import ee
import geemap
# 不再需要 import sys, import os (除非其他地方用到)
# 也不再需要任何 sys.path.append(...)

# 初始化GEE
ee.Initialize(project='geemap-441216') # 替换为您的项目ID

# Python会自动在当前文件夹下寻找模块，所以可以直接导入！
try:
    import z_flood_robust
    from z_flood_robust import zscore # 甚至可以这样导入
    zscore.calc_basemad # 尝试访问
    print("本地自定义模块 z_flood_robust 已成功导入！")
except ImportError as e:
    print(f"导入模块失败: {e}")
    print("请确保'z_flood_robust'文件夹与您的Jupyter笔记本在同一个目录下。")

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_7TDKVSyKvBdmMqW?ref=4i2o6


本地自定义模块 z_flood_robust 已成功导入！


# 1.导入依赖-数据管理模块

In [3]:
import pandas as pd
from datetime import datetime
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle #用于在图表中添加形状(例如矩形)
import re #正则表达式模块
from z_flood_robust import calc_basemedian, calc_basemad, calc_median_anomaly, calc_robust_zscore, calc_basemean,calc_basestd,calc_zscore
from z_flood_robust import mapFloods,floodPalette

from ipywidgets import Label #用于创建交互式控件



# 2.定义交互式地图以选择感兴趣区域-界面与交互模块

In [4]:
#解析用户点击地图时生成的坐标字符串，提取经纬度并返回
def parseClickedCoordinates(label):
  #利用正则表达式，从label.value提取出坐标：一个可能带有负号的浮点数 这里为【经度，纬度】
  #正则表达式： r'(?:-)?[0-9]+.[0-9]+'
  #r'表示原始字符，(?:-)是非捕获组，?:表示负号是可选的，[0-9]+表示匹配一个数字，+表示可以出现一次或多次，.匹配小数点，最后再次匹配一个或多个数字
  coords = [float(c) for c in re.findall(r'(?:-)?[0-9]+.[0-9]+', label.value)]
  coords.reverse() #反转为【纬度，经度】，符合GEE坐标格式
  return coords

#创建一个Lable，用于显示用户点击的目标
l = Label()
display(1)
#处理用户与地图的交互事件，当用户点击地图时，将点击的坐标存储到Label控件中
def handle_interaction(**kwargs):
  #kwargs包含交互事件的参数 kwargs.get('type'):获取事件类型为鼠标点击 kwargs.get('coordinates')：获取点击的坐标，转换为字符串并存储到Label控件中
  if kwargs.get('type') == 'click':
    l.value = str(kwargs.get('coordinates'))

print('请点击地图以选择你要监测的区域')
#创建交互式地图
Map = geemap.Map()
Map.on_interaction(handle_interaction)
Map

1

请点击地图以选择你要监测的区域


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], position='topright', transp…

# 3.定义几何范围并展示-界面与交互模块

## 1.确定研究区域

In [5]:
lon,lat = parseClickedCoordinates(l)
w,h = 0.3,0.3 #矩形宽度与高度（单位：度）

geometry = ee.Geometry.Polygon(
    [[[lon-w,lat-h],
     [lon-w,lat+h],
     [lon+w,lat+h],
     [lon+w,lat-h]]]
)

#将几何范围添加到地图
Map.addLayer(
    geometry,
    {'color':'red','fillColor':'00000000'},
    'AOI'
)
Map

Map(bottom=399568.0, center=[39.39269330108945, -0.38710869058407044], controls=(WidgetControl(options=['posit…

In [6]:
#USE MAP RECTANGLE 选择区域
#获取感兴趣区域（用于论文绘图)
roi_choose1 = Map.user_roi

if roi_choose1 is not None:
  #获取ROI类型
  roi_type = roi_choose1.type().getInfo()
  print(f"ROI 类型：{roi_type}")

  #如果是Polygon，获取坐标
  if roi_type == 'Polygon':
    coords = roi_choose1.coordinates().getInfo()
    print(f"ROI 坐标：{coords}")

ROI 类型：Polygon
ROI 坐标：[[[-0.441246, 39.33626], [-0.441246, 39.463603], [-0.340281, 39.463603], [-0.340281, 39.33626], [-0.441246, 39.33626]]]


# 4.过滤Sentinel-数据

In [7]:
# 所需日期
targdate = '2024-10-31'

# 
filters = [
    ee.Filter.listContains("transmitterReceiverPolarisation","VV"),
    ee.Filter.equals("instrumentMode","IW"),
    ee.Filter.equals("orbitProperties_pass","ASCENDING"),
    ee.Filter.date('2022-08-21', ee.Date(targdate).advance(1,'day'))
]

# 加载S1数据并计算Z分数
s1 = ee.ImageCollection("COPERNICUS/S1_GRD") \
    .filter(filters) \
    .filterBounds(roi_choose1) # !!! 使用统一的 analysis_aoi !!!
    

# 检查是否有影像满足条件
s1_size = s1.size().getInfo()
print(f"找到满足条件的 Sentinel-1 影像数量: {s1_size}")
if s1_size == 0:
    print("警告：未找到满足条件的影像，请检查时间范围、AOI 或轨道方向过滤器！")
print(s1.aggregate_array('system:time_start').map(lambda t: ee.Date(t).format()).getInfo())

band = 'VV'
flood_image = s1.mosaic().clip(roi_choose1).select(band)
Map.addLayer(flood_image, {'min':-25,'max':5}, 'Flood Image')




找到满足条件的 Sentinel-1 影像数量: 133
['2022-08-26T17:54:58', '2022-08-31T18:02:53', '2022-09-07T17:54:59', '2022-09-12T18:02:54', '2022-09-19T17:54:59', '2022-09-24T18:02:54', '2022-10-01T17:54:59', '2022-10-06T18:02:54', '2022-10-13T17:54:59', '2022-10-25T17:55:00', '2022-10-30T18:02:54', '2022-11-06T17:54:59', '2022-11-11T18:02:54', '2022-11-18T17:54:59', '2022-11-23T18:02:54', '2022-11-30T17:54:58', '2022-12-05T18:02:53', '2022-12-12T17:54:58', '2022-12-17T18:02:52', '2022-12-24T17:54:57', '2022-12-29T18:02:52', '2023-01-05T17:54:57', '2023-01-10T18:02:51', '2023-01-17T17:54:56', '2023-01-22T18:02:51', '2023-01-29T17:54:56', '2023-02-03T18:02:51', '2023-02-10T17:54:55', '2023-02-15T18:02:50', '2023-02-22T17:54:55', '2023-02-27T18:02:50', '2023-03-06T17:54:55', '2023-03-11T18:02:50', '2023-03-18T17:54:55', '2023-03-23T18:02:50', '2023-03-30T17:54:55', '2023-04-04T18:02:50', '2023-04-11T17:54:56', '2023-04-16T18:02:51', '2023-04-23T17:54:56', '2023-04-28T18:02:51', '2023-05-05T17:54:57', '202

# 5.自动阈值分割


In [8]:
#二值化+开运算

globalThreshold = ee.Number(-13.5)
globalWater = flood_image.lt(globalThreshold)
corrosion_kernel = ee.Kernel.circle(radius = 2)
eroded_Dark_targets = globalWater.focal_min(kernel = corrosion_kernel, iterations = 1).focal_max(kernel = corrosion_kernel, iterations = 1)
Map.addLayer(eroded_Dark_targets.selfMask(), {'palette':['red']}, 'Eroded Water')
Map

Map(bottom=199964.0, center=[39.37677199661635, -0.3622055053710938], controls=(WidgetControl(options=['positi…

# 6.边缘检测+缓冲区+外包矩形

## 6.1 边缘检测+缓冲区

In [9]:
# 定义方法需要的参数
connectedPixels = 100 # 长度计算连接像素数
edgeLength = 50 #水边缘的长度
edgeBuffer = 15 #边缘的缓冲区(单位m)
cannyThreshold = 1 #canny边缘检测的阈值
cannySigma = 1 #Canny 边缘检测中高斯滤波器的 Sigma 值
cannyLt = 0.05 #canny边缘检测的更限制的阈值

In [10]:
# Canny边缘检测
canny = ee.Algorithms.CannyEdgeDetector(
    image=eroded_Dark_targets,
    threshold=cannyThreshold,
    sigma=cannySigma
)

#获取边缘（排除强边缘 只要弱边缘 因强边缘会是噪声 即边缘强度小于阈值）
#connectedPixelCount()函数计算每个像素的相邻相连像素的数量，即边缘的长度
connected = canny.updateMask(canny).lt(cannyLt).connectedPixelCount(connectedPixels,True)

#新的像素数量图像中，每个像素值表示原始图像中，与其相连的、且边缘强度小于cannyLt的像素的数量
#让短边缘视为噪声
edges = connected.gte(edgeLength)

#基于边缘创建缓冲区，并将缓冲区内的像素设为1
#fastDistanceTransform():快速距离变换函数：计算每个像素到最近的非零像素（边缘像素）的距离
bufferEdges = edges.fastDistanceTransform(edgeBuffer).lte(edgeBuffer)
#将缓冲区外部分
edgeImage = flood_image.select(band).updateMask(bufferEdges)

edgeVis = {'palette' : 'yellow' , 'opacity' : 0.5}



# 6.2 边缘缓冲区生成外包矩形

In [11]:
print("正在将栅格斑块转为矢量")
bufferVectors = bufferEdges.selfMask().reduceToVectors(
    geometry = roi_choose1,
    scale = 30,
    geometryType = 'polygon',
    eightConnected = True,
    labelProperty = 'labels',
    maxPixels = 1e10
)
bufferVectors = bufferVectors
#.filter(ee.Filter.gt('count',1000/(30*30)))

print('正在为每个矢量计算外包矩形')
def getBoundingBox(feature):
   # feature.geometry() 获取当前要素的几何形状
  # .bounds() 计算该几何形状的最小外包矩形
  # ee.Feature() 用新的矩形几何体创建一个新的要素，并保留原始属性
  return ee.Feature(feature.geometry().bounds()) # Removed feature.properties

#将上述函数应用到集合中的每一个要素
boundingBoxes = bufferVectors.map(getBoundingBox)

# ---
# 步骤 4: 可视化外包矩形
# ---
# 为了只显示边框而不是实心矩形，我们创建一个空的图像，然后将矩形的边框“画”上去。
# 这比直接添加 FeatureCollection 并设置样式更稳健。
empty = ee.Image().byte() # 创建一个空的8位图像作为画布

# 使用 .paint() 函数绘制边框
outline = empty.paint(
    featureCollection=boundingBoxes, # 要绘制的要素集合
    color=1, # 边框的颜色，这里设为1，以便使用统一颜色
    width=2  # 边框的宽度（像素）
)

# 定义一个鲜艳的颜色方案来显示外包矩形
bboxVis = {'palette': 'FF00FF'} # 使用亮紫色 (Magenta)

print("将外包矩形添加到地图...")
Map.addLayer(outline, bboxVis, 'Detected Bounding Boxes')

# 将地图中心设置到您的研究区
Map.centerObject(roi_choose1, 11)
Map

正在将栅格斑块转为矢量
正在为每个矢量计算外包矩形
将外包矩形添加到地图...


Map(bottom=199964.0, center=[39.39991576443971, -0.39076350000047855], controls=(WidgetControl(options=['posit…

# 7.外包矩形筛选及提取

## 7.1 矩形筛选

In [12]:
import numpy as np

#筛选策略：基于剖面线“离群点百分比”的筛选法
def analyze_and_filter_bbox_by_outlier_percentage(feature):
    try:
        error_margin = ee.ErrorMargin(1) # 设置误差范围为1米
        bbox_geom = feature.geometry() # 获取外包矩形的几何形状
        center = bbox_geom.centroid(error_margin) # 计算外包矩形的中心点
        coords = ee.List(bbox_geom.coordinates().get(0)) # 获取外包矩形的坐标列表
        
        #获取矩形的三个角点
        p1, p2, p3 = ee.List(coords.get(0)), ee.List(coords.get(1)), ee.List(coords.get(2))
        
        # 客户端几何计算
        p1_coords, p2_coords, p3_coords = np.array(p1.getInfo()), np.array(p2.getInfo()), np.array(p3.getInfo())
        center_coords= np.array(center.coordinates().getInfo())
        vector_12, vector_23 = p2_coords - p1_coords, p3_coords - p2_coords  # 计算两个边的向量
        
        # ---新增:长宽比筛选---
        side1_length = np.linalg.norm(vector_12)
        side2_length = np.linalg.norm(vector_23)
        if side1_length < side2_length:
            side1_length, side2_length = side2_length, side1_length # 确保 side1_length 是较长的边
        #计算长宽比并检查
        aspect_ratio = side1_length / side2_length if side2_length != 0 else float('inf')
        if aspect_ratio > MAX_ASPECT_RATIO:
            return feature.set({
                'is_valid':ee.Number(0),
                'outelier_percentage':-1,
                'reason':f'AspectRatioTooHigh ({aspect_ratio:.1f}:1)'
            })
        # --结束--
        main_axis_vector = vector_12 if np.linalg.norm(vector_12) < np.linalg.norm(vector_23) else vector_23  # 选择较短的边作为主轴
        start_point = center_coords - main_axis_vector * 1.5
        end_point = center_coords + main_axis_vector * 1.5
        
        # 在GEE中创建剖面线
        long_line = ee.Geometry.LineString([start_point.tolist(), end_point.tolist()])
        profile_line = long_line.intersection(bbox_geom, error_margin)
        
        # 使用reduceRegion进行沿线采样
        sampled_dict = flood_image.select('VV').reduceRegion(
            reducer=ee.Reducer.toList(),
            geometry=profile_line,
            scale=10,
            maxPixels=1e10
        )
        
        sar_values = sampled_dict.get('VV').getInfo()
        if not sar_values or len(sar_values) < 3:
            return feature.set({
                'is_valid': ee.Number(0),
                'outlier_percentage': -1,
                'profile_line': profile_line,
                'reason': 'NotEnoughPoints'
            })

        # 客户端计算
        sar_array = np.array(sar_values)
        outlier_count = np.sum(sar_array < OUTLIER_SAR_THRESHOLD)
        total_count = len(sar_array)
        outlier_percentage = outlier_count / total_count

        is_valid_python = outlier_percentage >= MIN_OUTLIER_PERCENTAGE

        return feature.set({
            'is_valid': ee.Number(1) if is_valid_python else ee.Number(0), #不能直接设为True or False布尔型，因为与gee数据格式不兼容
            'outlier_percentage': outlier_percentage,
            'profile_line': profile_line,
            'reason': 'Passed' if is_valid_python else 'LowPercentage'
        })

    except Exception as e:
        return feature.set({
            'is_valid': ee.Number(0),
            'outlier_percentage': -1,
            'reason': f'Error: {str(e)}'
        })

# ==============================================================================
# 参数定义
# ==============================================================================
OUTLIER_SAR_THRESHOLD = -13.0
MIN_OUTLIER_PERCENTAGE = 0.1
MAX_ASPECT_RATIO = 5.0
# ==============================================================================
# 核心调试循环 (最终修复版)
# ==============================================================================
bboxes_list = boundingBoxes.toList(boundingBoxes.size())
processed_features_list = []
# [修复] 我们不再需要 all_profile_lines 列表，直接从最终集合中提取

print("\n--- [最终修复版 v2] 逐个检查每个矩形的处理过程 ---")
print("="*80)
print(f"{'Rect ID':<10} | {'Outlier %':<15} | {'Status':<15} | {'Reason/Details'}")
print("-"*80)

for i in range(bboxes_list.size().getInfo()):
    server_feature = ee.Feature(bboxes_list.get(i))
    client_processed_feature = analyze_and_filter_bbox_by_outlier_percentage(server_feature)

    # [保持不变] 打印主信息
    properties = client_processed_feature.getInfo()['properties']
    is_valid_flag = properties.get('is_valid', 0) == 1
    percentage = properties.get('outlier_percentage', -1)
    status = "PASSED" if is_valid_flag else "FAILED"
    print(f"{i:<10} | {f'{percentage:.2%}':<15} | {status:<15} | {properties.get('reason', 'N/A')}")

    # [保持不变] 打印失败的详细信息
    if not is_valid_flag and percentage != -1:
        profile_line = client_processed_feature.get('profile_line')
        if profile_line is not None:
            profile_line_geom = ee.Geometry(profile_line)
            sampled_dict_debug = flood_image.select('VV').reduceRegion(
                reducer=ee.Reducer.toList(),
                geometry=profile_line_geom,
                scale=10
            )
            values_debug_list = ee.List(sampled_dict_debug.get('VV')).getInfo()
            values_str = [f"{v:.2f}" for v in values_debug_list] if values_debug_list else ["No values sampled"]
            print(f"{'':<10} | {'':<15} | {'':<15} | RZ Values: [{', '.join(values_str)}]")

    processed_features_list.append(client_processed_feature)
print("="*80)

# ==============================================================================
# 后续处理和可视化 (最终修复版)
# ==============================================================================
analyzed_bboxes = ee.FeatureCollection(processed_features_list)
filtered_boundingBoxes = analyzed_bboxes.filter(ee.Filter.eq('is_valid', 1))

print(f"\n最终结果：")
print(f"筛选前矩形总数: {analyzed_bboxes.size().getInfo()}")
print(f"筛选后（离群点百分比 >= {MIN_OUTLIER_PERCENTAGE:.0%}）的矩形数量: {filtered_boundingBoxes.size().getInfo()}")

# --- 可视化 ---

# [核心修复] 定义一个函数，将Feature的几何体替换为其'profile_line'属性
def extract_profile_as_feature(feature):
    # 从输入 feature 中获取 profile_line 几何体
    profile_geom = ee.Geometry(feature.get('profile_line'))
    # 返回一个新的 Feature，它的几何体是 profile_line，并且不包含任何属性
    # 这样可以避免属性继承带来的问题
    return ee.Feature(profile_geom)

# -- 可视化所有剖面线 (用于调试) --
# 使用 .map() 应用上面的函数，确保我们得到一个 FeatureCollection of LineStrings
all_profiles_fc = analyzed_bboxes.map(extract_profile_as_feature)
Map.addLayer(ee.Image().byte().paint(all_profiles_fc, 0, 1), {'palette': 'FF0000'}, 'All Profile Lines (DEBUG)') # 红色

# -- 可视化通过筛选的矩形和剖面线 --
if filtered_boundingBoxes.size().getInfo() > 0:
    # 可视化矩形边框 (这部分没问题)
    filtered_outline = ee.Image().byte().paint(filtered_boundingBoxes, 0, 2)
    Map.addLayer(filtered_outline, {'palette': '00008B'}, 'Filtered BBoxes (PASSED)') # 绿色

    # [核心修复] 使用同样的方法提取通过筛选的剖面线
    valid_profile_lines_fc = filtered_boundingBoxes.map(extract_profile_as_feature)
    valid_profile_lines = ee.Image().byte().paint(valid_profile_lines_fc, 0, 2)
    Map.addLayer(valid_profile_lines, {'palette': '00FFFF'}, 'Valid Profile Lines (PASSED)') # 青色
else:
    print("\n没有找到有效的矩形。")

Map.centerObject(roi_choose1, 11)
Map





--- [最终修复版 v2] 逐个检查每个矩形的处理过程 ---
Rect ID    | Outlier %       | Status          | Reason/Details
--------------------------------------------------------------------------------
0          | 100.00%         | PASSED          | Passed
1          | 93.33%          | PASSED          | Passed
2          | 75.00%          | PASSED          | Passed
3          | 66.15%          | PASSED          | Passed
4          | 73.13%          | PASSED          | Passed
5          | 47.27%          | PASSED          | Passed
6          | -100.00%        | FAILED          | NotEnoughPoints
7          | 63.29%          | PASSED          | Passed
8          | 76.71%          | PASSED          | Passed
9          | 53.06%          | PASSED          | Passed
10         | 59.02%          | PASSED          | Passed
11         | 45.45%          | PASSED          | Passed
12         | 12.50%          | PASSED          | Passed

最终结果：
筛选前矩形总数: 13
筛选后（离群点百分比 >= 10%）的矩形数量: 12


Map(bottom=199920.0, center=[39.39991576443971, -0.39076350000047855], controls=(WidgetControl(options=['posit…

## 7.2 均值剖面线提取


In [13]:
from scipy.ndimage import map_coordinates
import matplotlib.pyplot as plt

def create_robust_profile(feature, sar_image_ee):
    """
    为单个矩形特征生成一条平滑且鲁棒的均值剖面线

    Args:
        feature (ee.Feature):包含"profile_line'属性的经筛选的有效矩形 
        sar_image_ee (ee.Image): 用于采样的原始SAR影像
        
    Returns:
        dict:包含剖面线信息的字典
    """
    try:
        profile_line = ee.Geometry(feature.get('profile_line'))
        line_coords = np.array(profile_line.coordinates().getInfo()) # 获取剖面线坐标
        start_point, end_point = line_coords[0], line_coords[1] # 起点和终点坐标
        
        # 计算剖面线长度和缓冲区半径
        line_length_meters = profile_line.length().getInfo()
        buffer_radius_L = line_length_meters * 0.2 # 缓冲区长度为剖面线长度的20%
        
        # 建立缓冲区
        profile_buffer = profile_line.buffer(buffer_radius_L)
        
        #---2.转换缓冲区内SAR像素到客户端---
        # sar_patch_up是包含缓冲区像素值的一个numpy数组，patch_info是转换的元数据
        sar_patch_np, patch_info = geemap.ee_to_numpy(
            sar_image_ee,
            region = profile_buffer.bounds(), 
            scale = 10,
        )
        
        if sar_patch_np is None or sar_patch_np.size < 2:  # 确保有足够的像素点
            raise ValueError("无法获取SAR图像数据，可能是区域内没有数据。")
        
        sar_patch_np = sar_patch_np.squeeze() # 去除单维度 将 (1, H, W) 转为 (H, W)
        
        # ---3.在客户端进行坐标转换和采样---
        # 获取地理转换参数
        affine_transform = patch_info['transform'] # 仿射变换矩阵，是将像素坐标转换为地理坐标的矩阵
        inv_transform = ~affine_transform # 计算逆变换矩阵 这个矩阵可以将地理坐标反算为像素坐标
        
        # 将剖面线的地理坐标转换为像素坐标
        px_start, py_start = inv_transform * (start_point[0], start_point[1])
        px_end, py_end = inv_transform * (end_point[0], end_point[1])
        
        # 计算剖面线的向量和垂直方向法向量
        vec_line = np.array([px_end - px_start, py_end - py_start]) # 剖面线向量
        vec_line_normalized = vec_line / np.linalg.norm(vec_line) # 归一化剖面线向量
        vec_perp = np.array([-vec_line_normalized[1], vec_line_normalized[0]]) # 像素坐标法向量（垂直方向）
        
        # 计算缓冲区半径对应的像素距离
        pixel_size = affine_transform[0] # 像素大小
        buffer_radius_pixels = buffer_radius_L / pixel_size # 缓冲区半径对应的像素距离
        
        # ---4.生成均值剖面线 ---
        num_steps_along_line = int(line_length_meters / pixel_size) # 沿剖面线的采样点数量
        num_steps_perp = int(buffer_radius_pixels * 2) # 垂直方向的采样点数
        
        robust_profile_values= []
        
        # 沿着剖面线的每个点进行迭代
        for i in range(num_steps_along_line):
            # 计算当前点在剖面线上的像素坐标:起点 + i * 单位向量
            current_point_on_line = np.array([px_start, py_start]) + i * vec_line_normalized
            
            perp_values = []
            # 沿着垂直方向进行采样 
            for j in range(-num_steps_perp // 2, num_steps_perp // 2): #整数除法 两边各占l的0.2
                # 计算垂直方向上的采样点坐标，即当前点 + j * 垂直法向量
                sample_point = current_point_on_line + j * vec_perp
                
                # 使用线性插值(order=1)获取SAR值
                # map_coordinates 需要的座標格式是 [[y1, y2, ...], [x1, x2, ...]]
                # 在 sar_patch_np 這塊 SAR 影像上，針對 sample_point 這個浮點數像素座標 (x, y)，使用線性內插的方法來精確計算其對應的雷達反射強度值。如果計算點超出了影像邊界，則自動使用最邊緣的像素值。
                value = map_coordinates(sar_patch_np, [[sample_point[1]], [sample_point[0]]], order=1, mode='nearest')
                perp_values.append(value[0]) #将垂直方向的采样值存储起来

            # 計算當前剖面線點的平均值
            robust_profile_values.append(np.mean(perp_values))

        return {
            'original_profile_coords': line_coords,
            'robust_profile_values': robust_profile_values,
            'buffer_geom': profile_buffer
        }
        
    except Exception as e:
        import traceback
        print(f"Error！Abnormal in create_robust_profile: {e}")
        traceback.print_exc()
        return None


In [14]:
# 主执行流程
print("\n--- [步驟 8] 開始生成平滑且魯棒的均值剖面線 ---")

if 'filtered_boundingBoxes' not in globals() or filtered_boundingBoxes.size().getInfo() == 0:
    print("沒有有效的矩形可供處理。請確保前面的步驟已成功完成並且有通過篩選的矩形。")
else:
    valid_bboxes_list = filtered_boundingBoxes.toList(filtered_boundingBoxes.size())
    num_valid_bboxes = filtered_boundingBoxes.size().getInfo()
    print(f"找到 {num_valid_bboxes} 個有效的矩形，正在處理...")
    
    all_robust_profiles = []
    
        # 為了演示，我們只處理前5個矩形，避免執行時間過長
    # 在您的最終版本中，可以改為 `range(num_valid_bboxes)`
    num_to_process = min(5, num_valid_bboxes)
    
    for i in range(num_to_process):
        print(f"\n--- 正在處理矩形 {i+1}/{num_to_process} ---")
        feature = ee.Feature(valid_bboxes_list.get(i))
        
        # 調用核心函式生成均值剖面線，使用本流程中的 flood_image
        profile_data = create_robust_profile(feature, flood_image)
        
        if profile_data:
            all_robust_profiles.append(profile_data)
            print(f"成功生成均值剖面線，長度為: {len(profile_data['robust_profile_values'])} 點。")
            # 將生成的緩衝區添加到地圖上進行驗證
            Map.addLayer(profile_data['buffer_geom'], {'color': 'cyan'}, f'Robust Profile Buffer {i+1}', False)
        else:
            print("生成均值剖面線失敗。")

    print(f"\n--- 處理完成 ---")
    print(f"共成功生成了 {len(all_robust_profiles)} 條魯棒剖面線。")

    # --- 可視化其中一條剖面線作為示例 ---
    if all_robust_profiles:
        print("\n繪製第一條生成的均值剖面線作為示例：")
        
        # 提取第一條剖面線的資料
        example_profile = all_robust_profiles[0]
        values = example_profile['robust_profile_values']
        
        plt.figure(figsize=(12, 6))
        plt.plot(values, marker='.', linestyle='-', color='b')
        plt.title('Example of a Robust Mean Profile Line')
        plt.xlabel('Distance along Profile (in pixels)')
        plt.ylabel('Mean SAR Backscatter (dB)')
        plt.grid(True)
        plt.show()

Map


--- [步驟 8] 開始生成平滑且魯棒的均值剖面線 ---
找到 12 個有效的矩形，正在處理...

--- 正在處理矩形 1/5 ---
Error！Abnormal in create_robust_profile: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices
生成均值剖面線失敗。

--- 正在處理矩形 2/5 ---


Traceback (most recent call last):
  File "C:\Users\Administrator\AppData\Local\Temp\ipykernel_10196\1884381626.py", line 42, in create_robust_profile
    affine_transform = patch_info['transform'] # 仿射变换矩阵，是将像素坐标转换为地理坐标的矩阵
                       ~~~~~~~~~~^^^^^^^^^^^^^
IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices


Error！Abnormal in create_robust_profile: too many values to unpack (expected 2)
生成均值剖面線失敗。

--- 正在處理矩形 3/5 ---


Traceback (most recent call last):
  File "C:\Users\Administrator\AppData\Local\Temp\ipykernel_10196\1884381626.py", line 29, in create_robust_profile
    sar_patch_np, patch_info = geemap.ee_to_numpy(
    ^^^^^^^^^^^^^^^^^^^^^^^^
ValueError: too many values to unpack (expected 2)


Error！Abnormal in create_robust_profile: too many values to unpack (expected 2)
生成均值剖面線失敗。

--- 正在處理矩形 4/5 ---


...

Error！Abnormal in create_robust_profile: too many values to unpack (expected 2)
生成均值剖面線失敗。

--- 正在處理矩形 5/5 ---


Traceback (most recent call last):
  File "C:\Users\Administrator\AppData\Local\Temp\ipykernel_10196\1884381626.py", line 29, in create_robust_profile
    sar_patch_np, patch_info = geemap.ee_to_numpy(
    ^^^^^^^^^^^^^^^^^^^^^^^^
ValueError: too many values to unpack (expected 2)


Error！Abnormal in create_robust_profile: too many values to unpack (expected 2)
生成均值剖面線失敗。

--- 處理完成 ---
共成功生成了 0 條魯棒剖面線。


Traceback (most recent call last):
  File "C:\Users\Administrator\AppData\Local\Temp\ipykernel_10196\1884381626.py", line 29, in create_robust_profile
    sar_patch_np, patch_info = geemap.ee_to_numpy(
    ^^^^^^^^^^^^^^^^^^^^^^^^
ValueError: too many values to unpack (expected 2)


Map(bottom=399547.0, center=[39.398264899543584, -0.3946495056152344], controls=(WidgetControl(options=['posit…